In [1]:
import os
import sys
import numpy as np
import pandas as pd
from collections import defaultdict

In [2]:
df_data = pd.read_csv('DeepLINK/Real_data_analyses/human_microbiome/data/microbiome_data_common.csv')
df_data.head()

,X195,X197,X199,X287,X294,X316,X470,X539,X545,X546,...,X1905288,X1907578,X1912856,X1915310,X1934254,X1944646,X2025876,X2026240,X2051905,real_y
0,0.000899,0.000000e+00,2.830000e-06,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000,0.000065,...,8.850000e-111,0.000000,0.000095,0.000000e+00,0.000000,0.000017,0.018672,0.000000e+00,0.000000e+00,0
1,0.000075,1.470000e-45,0.000000e+00,1.680000e-07,2.180000e-08,0.0,0.000000,0.000000,0.000005,0.000000,...,2.890000e-33,0.000000,0.000040,5.270000e-249,0.000002,0.000002,0.025775,0.000000e+00,2.790000e-196,0
2,0.000275,3.560000e-05,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000047,0.000002,0.000000,0.000000,...,0.000000e+00,0.000000,0.000094,0.000000e+00,0.000000,0.000026,0.014592,0.000000e+00,0.000000e+00,0
3,0.000133,2.900000e-190,6.770000e-07,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000017,0.000000e+00,0.000000,0.000002,0.006446,0.000000e+00,1.390000e-64,0
4,0.001387,2.857100e-04,0.000000e+00,1.510000e-06,0.000000e+00,0.0,0.000002,0.000002,0.000000,0.000873,...,7.440000e-06,0.000002,0.000062,1.720000e-05,0.000006,0.000014,0.028903,4.410000e-07,1.550000e-05,0


In [3]:
feature_names = df_data.columns[:-1]
feature_names

Index(['X195', 'X197', 'X199', 'X287', 'X294', 'X316', 'X470', 'X539', 'X545',
       'X546',
       ...
       'X1902136', 'X1905288', 'X1907578', 'X1912856', 'X1915310', 'X1934254',
       'X1944646', 'X2025876', 'X2026240', 'X2051905'],
      dtype='object', length=274)

In [4]:
def summarize(explanation_res_path):
    df_exp = pd.read_csv(explanation_res_path, header=None)
    df_exp.columns = ['sample_id', 'feature_id', 'importance']

    single_ranks = [[] for _ in range(len(feature_names))]
    interaction_ranks = defaultdict(list)


    sample_num = df_exp['sample_id'].max() + 1
    for i in range(sample_num):
        sub_df = df_exp[df_exp['sample_id'] == i]
        sub_df = sub_df.reset_index(drop=True)
        for fi, row in sub_df.iterrows():
            feature_id = row['feature_id']
            feature_id = [int(x) for x in feature_id.split('-')]
            if len(feature_id) == 1:
                single_ranks[feature_id[0]].append(fi + 1)
            else:
                feature_id = sorted(feature_id)
                for p in range(len(feature_id)):
                    for q in range(p + 1, len(feature_id)):
                        interaction_ranks[(feature_id[p], feature_id[q])].append(fi + 1)

    avg_single_ranks = []
    for fi, ranks in enumerate(single_ranks):
        avg_single_ranks.append(np.mean(ranks))

    metric_interaction_ranks = {}
    for pair, ranks in interaction_ranks.items():
        metric_interaction_ranks[pair] = (np.sum(ranks) + (len(feature_names) + 1) * (sample_num - len(ranks))) / sample_num

    # avg_ranks = [(feature_names[i], avg_single_ranks[i]) for i in range(len(feature_names))] + [(f'{feature_names[pair[0]]} - {feature_names[pair[1]]}', metric_interaction_ranks[pair]) for pair in metric_interaction_ranks.keys()]
    # sorted_avg_ranks = sorted(avg_ranks, key=lambda x: x[1])

    # df_result = pd.DataFrame(
    #     {
    #         'feature': [pair[0] for pair in sorted_avg_ranks],
    #         'avg_rank': [pair[1] for pair in sorted_avg_ranks],
    #     }
    # )
    # return df_result

    single_sorted_idx = np.argsort(avg_single_ranks)
    df_single = pd.DataFrame(
        {
            'feature': [feature_names[i] for i in single_sorted_idx],
            'avg_rank': [avg_single_ranks[i] for i in single_sorted_idx],
            'h_rank': [np.min(single_ranks[i]) if len(single_ranks[i]) > 0 else len(feature_names) for i in single_sorted_idx],
            'l_rank': [np.max(single_ranks[i]) if len(single_ranks[i]) > 0 else len(feature_names) for i in single_sorted_idx],
        }
    )

    sorted_metric_interaction_ranks = sorted(metric_interaction_ranks.items(), key=lambda x: x[1])
    df_interaction = pd.DataFrame(
        {
            'feature': [f'{feature_names[pair[0]]} - {feature_names[pair[1]]}' for pair, _ in sorted_metric_interaction_ranks],
            'avg_rank': [np.mean(interaction_ranks[pair]) for pair, _ in sorted_metric_interaction_ranks],
            'count':  [len(interaction_ranks[pair]) for pair, _ in sorted_metric_interaction_ranks],
            'h_rank': [np.min(interaction_ranks[pair]) for pair, _ in sorted_metric_interaction_ranks],
            'l_rank': [np.max(interaction_ranks[pair]) for pair, _ in sorted_metric_interaction_ranks],
        }
    )

    return df_single, df_interaction


def run_summarize(explanation_res_path):
    parent_folder = os.path.dirname(explanation_res_path)
    df_single, df_interaction = summarize(explanation_res_path)
    df_single.to_csv(os.path.join(parent_folder, 'sorted_single_features.csv'), index=False)
    df_interaction.to_csv(os.path.join(parent_folder, 'sorted_interaction_features.csv'), index=False)


run_summarize('explanations/human_microbiome_best/top_3/explanation.csv')
run_summarize('explanations/human_microbiome_best/top_5/explanation.csv')
run_summarize('explanations/human_microbiome_best/top_10/explanation.csv')